In [1]:
import pandas as pd
import numpy as np
import mibian
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import py_vollib 
import py_vollib.black_scholes.implied_volatility as biv
#import py_vollib.black.implied_volatility as biv
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, vega

In [2]:
#OptionList = pd.read_csv("nifty_311221_070122_1min.csv",index_col=0)
OptionList = pd.read_csv("nifty_311221_190122_1min.csv",index_col=0)

In [ ]:
# see reference: https://github.com/yassinemaaroufi/MibianLib
def calculateIV(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']
    price = row['opt_price']
    if row['instrument_type'] == 'CE':
        bs = mibian.BS([s,k,r,t],callPrice=price)
    elif row['instrument_type'] == 'PE':
        bs = mibian.BS([s,k,r,t],putPrice=price)
    return bs.impliedVolatility

def calculateDelta(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']
    v = row['IV']
    bs = mibian.BS([s,k,r,t],volatility=v)
    if row['instrument_type'] == 'CE':
        res = bs.callDelta
    elif row['instrument_type'] == 'PE':
        res = bs.putDelta
    return res

def calculateVega(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']
    v = row['IV']
    bs = mibian.BS([s,k,r,t],volatility=v)
    return bs.vega

In [3]:
# see reference: http://vollib.org/documentation/python/1.0.2/index.html
# see reference: https://github.com/vollib/py_vollib
def calculateIV_py_vollib(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']/252
    price = row['opt_price']
    if row['instrument_type'] == 'CE':
        iv = biv.implied_volatility(price, s, k, t, r, 'c')
    elif row['instrument_type'] == 'PE':
        iv = biv.implied_volatility(price, s, k, t, r, 'p')
    return iv

def calculateDelta_py_vollib(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']/252
    v = row['IV']
    if row['instrument_type'] == 'CE':
        res = delta('c',s,k,t,r,v)
    elif row['instrument_type'] == 'PE':
        res = delta('p',s,k,t,r,v)
    return res

def calculateVega_py_vollib(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']
    v = row['IV']
    vega_ = vega('c',s,k,t,r,v)
    return vega_

In [4]:
Calls = OptionList[OptionList['instrument_type']=='CE']
print(Calls.count()[0])
Calls = Calls[Calls['fut_price'] - Calls['strike'] <= Calls['opt_price']]
print(Calls.count()[0])

346687
320496


In [5]:
%%time
#Calls['IV'] = Calls.apply(calculateIV,axis=1)
Calls['IV'] = Calls.apply(calculateIV_py_vollib,axis=1)
#Calls['delta'] = Calls.apply(calculateDelta,axis=1)
Calls['delta'] = Calls.apply(calculateDelta_py_vollib,axis=1)
#Calls['vega'] = Calls.apply(calculateVega,axis=1)
Calls['vega'] = Calls.apply(calculateVega_py_vollib,axis=1)
Calls.to_csv('nifty_311221_190122_1min_calls_py_vollib.csv')

D:\Anaconda3\lib\site-packages\py_vollib\ref_python\black_scholes\__init__.py:87: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / denominator











































































































































































































































































































































































Wall time: 24.6 s


In [6]:
%%time
Calls['IV'] = Calls.apply(calculateIV_py_vollib,axis=1)

Wall time: 9.88 s


In [7]:
%%time
Calls['delta'] = Calls.apply(calculateDelta_py_vollib,axis=1)






















































































































































































Wall time: 5.64 s


In [8]:
%%time
Calls['vega'] = Calls.apply(calculateVega_py_vollib,axis=1)






















































































































































































Wall time: 5.73 s


In [9]:
Calls

date  opt_price  volume_x         opt_symbol  \
0       2021-12-31 09:15:00+05:30     231.50       0.0  NIFTY22JAN17450CE   
1       2021-12-31 09:15:00+05:30     544.95       0.0  NIFTY22JAN16950CE   
5       2021-12-31 09:15:00+05:30      60.50       0.0  NIFTY22JAN17950CE   
6       2021-12-31 09:15:00+05:30      50.70       0.0  NIFTY22JAN18050CE   
7       2021-12-31 09:15:00+05:30       4.25       0.0  NIFTY22JAN19600CE   
...                           ...        ...       ...                ...   
683711  2022-01-19 16:30:00+05:30    1976.45       0.0  NIFTY22JAN16000CE   
683712  2022-01-19 16:30:00+05:30       2.75       0.0  NIFTY22JAN19700CE   
683715  2022-01-19 16:30:00+05:30       3.00       0.0  NIFTY22JAN19550CE   
683852  2022-01-19 16:39:00+05:30       2.95       0.0  NIFTY22JAN19750CE   
683854  2022-01-19 16:39:00+05:30    1976.45       0.0  NIFTY22JAN16000CE   

        fut_price  volume_y     fut_symbol   name               expiry  \
0         17311.0  174100.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
1         17311.0  174100.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
5         17311.0  174100.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
6         17311.0  174100.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
7         17311.0  174100.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
...           ...       ...            ...    ...                  ...   
683711    17975.5       0.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
683712    17975.5       0.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
683715    17975.5       0.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
683852    17975.5       0.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   
683854    17975.5       0.0  NIFTY22JANFUT  NIFTY  2022-01-27 15:30:30   

         strike instrument_type                   ts  days_to_expiry  \
0       17450.0              CE  2021-12-31 09:15:00       27.260764   
1       16950.0              CE  2021-12-31 09:15:00       27.260764   
5       17950.0              CE  2021-12-31 09:15:00       27.260764   
6       18050.0              CE  2021-12-31 09:15:00       27.260764   
7       19600.0              CE  2021-12-31 09:15:00       27.260764   
...         ...             ...                  ...             ...   
683711  16000.0              CE  2022-01-19 16:30:00        7.958681   
683712  19700.0              CE  2022-01-19 16:30:00        7.958681   
683715  19550.0              CE  2022-01-19 16:30:00        7.958681   
683852  19750.0              CE  2022-01-19 16:39:00        7.952431   
683854  16000.0              CE  2022-01-19 16:39:00        7.952431   

              IV     delta        vega  
0       0.129723  0.434036  341.824831  
1       0.148551  0.675718  330.831796  
5       0.115278  0.174363  350.282377  
6       0.119119  0.147456  350.024870  
7       0.167532  0.012999  345.187455  
...          ...       ...         ...  
683711  0.247791  0.996160  177.084300  
683712  0.224188  0.011321  199.357870  
683715  0.210348  0.012958  199.885052  
683852  0.231732  0.011714  198.881558  
683854  0.247888  0.996160  177.014754  

[320496 rows x 16 columns]

In [11]:
Calls.to_csv('nifty_311221_190122_1min_calls_py_vollib.csv')